In [1]:
# -*- config : utf-8 -*-
import numpy as np
import tensorflow as tf
import sys
import time
import warnings

# 경고 메시지 무시
warnings.filterwarnings(action='ignore')

In [2]:
# --- 설정 변수 정의 ---
# 작업 디렉토리 정의
# 학습 데이터 파일명
noise_list_file = "/src/hyebin/esnoise/noise_spec_data.npy"
# 라벨 데이터 파일명
noise_label_list_file = "/src/hyebin/esnoise/noise_label_data.npy"
# 진동 분석 모델 디렉토리명
modeldir = "/src/hyebin/model/NOISE-EDGE-SPEC-01"
# tensorboard 용 log 디렉토리명
tblogdir = "/src/hyebin/logs"

# 학습 데이터 설정
# 학습할 데이터의 개수
training_count = 2000

# 전결합층 설정
# 전결합층 첫번째 레이어 unit 개수
layer1_unit_count = 256
# 전결합층 dropout 비율
dropout_rate = 0.4

# 최적화 함수 설정
# 최적화 함수 학습률
learning_rate = 0.0001

# 학습 수행 설정
# 최대 학습 수행 회수
epoch_count = 5000
# 학습 중단 시킬, 최소 loss값
loss_limit = 0.0001

In [3]:
# --- 학습 데이터 로딩 ---
# 데이터 구조
# noise_list : 학습용 데이터(이미지 개수, 48, 64, 3)
# noise_label_list : 학습용 라벨 데이터(이미지 개수, 18)
print("load noise data:", noise_list_file)
noise_list = np.load(noise_list_file, allow_pickle=True)
print("load complete")

print("load noise label data:", noise_label_list_file)
org_noise_label_list = np.load(noise_label_list_file, allow_pickle=True)

# one hot encoding
noise_label_list = []
unit_matrix = np.identity(18)
for index in org_noise_label_list:
    one_hot = unit_matrix[index:index+1][0]
    noise_label_list.append(one_hot)

noise_label_list = np.array(noise_label_list)

print("load complete")

load noise data: /src/hyebin/esnoise/noise_spec_data.npy
load complete
load noise label data: /src/hyebin/esnoise/noise_label_data.npy
load complete


In [4]:
# --- VGG Net 신경망 구성 ---
#     입력층 : 48*64 의 Spectrogram 전처리 데이터
#     특징 추출층 : 2개 합성곱 필터, 1개의 필터를 2회 수행 
#     전 결합층 : 3개의 은닉층(층당 유닛 1024개) → 단층에서 발생하는 xor 문제 회피, 
#                1개의 출력층(유닛 5개)

tf.reset_default_graph()

# dropout 사용 여부
dropout_rate = tf.placeholder_with_default(0, shape=[], name="dropout_rate")

# --- 입력층 ---
# Spectrogram
x = tf.placeholder(tf.float32, [None, 48, 64, 3], name="in")

# --- 특징 추출층 ---
conv11 = tf.layers.conv2d(
    inputs=x,
    filters=32,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

conv12 = tf.layers.conv2d(
    inputs=conv11,
    filters=32,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

# 풀링으로 사이즈가 줄어듦
# Spectrogram : 48 * 64 → 24 * 32
pool1 = tf.layers.max_pooling2d(
    inputs=conv12,
    pool_size=[2, 2],
    strides=2)

conv21 = tf.layers.conv2d(
    inputs=pool1,
    filters=64,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

conv22 = tf.layers.conv2d(
    inputs=conv21,
    filters=64,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

# 풀링으로 사이즈가 줄어듦
# Spectrogram : 24 * 32 → 12 * 16
pool2 = tf.layers.max_pooling2d(
    inputs=conv22,
    pool_size=[2, 2],
    strides=2)

# --- 전 결합층 ---
# 데이터 행렬 평탄화(flatten)
# Spectrogram : 6 * 8(이미지사이즈) * 64(합성곱 필터 수)
input_cnt = 12 * 16 * 64
pool_flat = tf.reshape(pool2, [-1, input_cnt])

dense1 = tf.layers.dense(inputs=pool_flat, units=layer1_unit_count, activation=tf.nn.relu)
dropout1 = tf.layers.dropout(inputs=dense1, rate=dropout_rate)

dense2 = tf.layers.dense(inputs=dropout1, units=layer1_unit_count, activation=tf.nn.relu)
dropout2 = tf.layers.dropout(inputs=dense2, rate=dropout_rate)

dense3 = tf.layers.dense(inputs=dropout2, units=layer1_unit_count, activation=tf.nn.relu)
dropout3 = tf.layers.dropout(inputs=dense3, rate=dropout_rate)

dense4 = tf.layers.dense(inputs=dropout3, units=layer1_unit_count, activation=tf.nn.relu)
dropout4 = tf.layers.dropout(inputs=dense4, rate=dropout_rate)

netout = tf.layers.dense(inputs=dropout4, units=18)
softout = tf.nn.softmax(netout, name="out")

# --- loss 함수 정의 ---
# 평균 제곱 오차 함수 사용
y = tf.placeholder(tf.float32, [None, 18])
loss = tf.reduce_mean(tf.reduce_sum(tf.square(softout - y), 1))
loss_summ = tf.summary.scalar("loss", loss)

# --- loss 함수의 최적화 함수 정의 ---
train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

W1227 14:33:54.521958 140558034323264 deprecation.py:323] From <ipython-input-4-77bbbb81e3e6>:22: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W1227 14:33:54.527358 140558034323264 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1227 14:33:54.812944 140558034323264 deprecation.py:323] From <ipython-input-4-77bbbb81e3e6>:36: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W1227 14:33:54.954103 140558034323264 deprecation.py:323] F

In [5]:
# --- 학습 수행 및 모델 저장---
with tf.Session() as sess:

    # tensorboard 용 로깅 정의    
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(tblogdir)
    writer.add_graph(sess.graph)
    
    # 신경망 가중치(weight) 초기화
    sess.run(tf.global_variables_initializer())
    
    # 전체 데이터에 대해 5000번 수행
    # loss가 0.001 이하일때, 학습 중단
    start_time = time.time()
    for epoch in range(epoch_count):
        
        # 학습용 데이터와 라벨
        # 랜덤 데이터를 training_count 만큼 가져옴
        base_index = np.random.randint(noise_label_list.shape[0] - training_count)
        train_x = noise_list[base_index:base_index + training_count]
        train_y = noise_label_list[base_index:base_index + training_count]

        # 학습 수행
        _train, _loss_summ, _loss = sess.run([train, loss_summ, loss], feed_dict={x:train_x, y:train_y, dropout_rate:0.4})
        writer.add_summary(summary = _loss_summ, global_step = epoch)
        
        if epoch % 100 == 0:
            print("index=", epoch, "  loss=", _loss)
            
        if _loss < loss_limit:
            print("index=", epoch, "  loss=", _loss)
            break
            
    end_time = time.time()
    print("running time:", end_time - start_time)
    
    # --- 학습 모델 저장 ----
    print("save model")
    builder = tf.saved_model.builder.SavedModelBuilder(modeldir)
    signature = tf.saved_model.predict_signature_def(inputs={"in":x}, outputs={"out":softout})
    builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING], signature_def_map={"serving_default":signature})
    builder.save()
    print("complete")

index= 0   loss= 0.9444451
index= 100   loss= 0.37848797
index= 200   loss= 0.12172048
index= 300   loss= 0.07446921
index= 400   loss= 0.06009061
index= 500   loss= 0.06715868
index= 600   loss= 0.0037801175
index= 700   loss= 0.0008391685
index= 800   loss= 0.0005388473
index= 900   loss= 0.00045083745
index= 1000   loss= 0.00033395382
index= 1100   loss= 0.00029730875


W1227 14:42:33.562742 140558034323264 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:201: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


index= 1199   loss= 9.805294e-05
running time: 516.5799572467804
save model
complete
